In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import fnmatch
import os
from matplotlib import colors
import matplotlib.pyplot as plt

2021-12-13 21:47:51.166399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-13 21:47:51.166415: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from sklearn.model_selection import train_test_split

## Read Input Data

In [3]:
def load_data(path_to_file,maze_size):
    X = []
    y = []
    f = open(path_to_file,'rb')
    
    try:
        while True:
            x1 = np.load(f).reshape(maze_size,maze_size)
            y1 = np.load(f)
            
            if(len(y1)<4):
                print(y1)
                continue
            X.append(x1)
            y.append(y1)
    except:
            # print("eof")
            pass
    return X,y
    

In [4]:
def read_dataset(maze_size):
    final_X = []
    final_y = []
    dirs = [d for d in os.listdir("./") if fnmatch.fnmatch(d,'dataset*')]
    # print(dirs)
    # dirs = dirs[-1:-2:-1]
    print(dirs)
    for directory in dirs:
        for f in os.listdir("./"+directory+"/"):
            if fnmatch.fnmatch(f,'data*'):
                X,y = load_data("./"+directory+"/"+f,maze_size)
                # print(y.shape,type(y))
                if(len(X)==len(y)):
                # print(len(X))
                    final_X += X
                    final_y += y
                else:
                    print("wrong inputs in :","./"+directory+"/"+f)
    
    return np.array(final_X),final_y

In [339]:
X,y = read_dataset(4)

['dataset9', 'dataset3', 'dataset', 'dataset4', 'dataset0', 'dataset7', 'dataset5', 'dataset2', 'dataset8', 'dataset10', 'dataset6', 'dataset1']


In [340]:
y = np.asarray(y)

In [341]:
y.shape

(1623551, 4)

In [342]:
X.shape

(1623551, 4, 4)

In [343]:
## Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [344]:
# train_in = np.reshape(X_train, (-1,4,4))
# test_in = np.reshape(X_test, (-1,4,4))
# train_out = np.reshape(y_train, (-1,4,1))
# test_out = np.reshape(y_test, (-1,4,1))
# print(train_in.shape,test_in.shape, type(train_in))

## Create Dummy data

In [30]:
temp = np.asarray([[[[1,2],
                     [2,3],
                     [5,4]],
                    
                     [[2,3],
                      [3,4],
                      [8,9]], 
                    
                     [[1,2],
                      [2,3],
                      [5,4]]]])

In [31]:
temp.shape

(1, 3, 3, 2)

In [70]:
dim=3
arr1 = np.empty([3, 3, 1])
arr2 = np.empty([3, 3, 1])
arr3 = np.empty([3,3,2])
for i in range(dim):
    for j in range(dim):
            arr3[i][j][0] = arr1[i][j]
            arr3[i][j][1] = arr2[i][j]

In [113]:
arr1

array([[[6.92263573e-310],
        [5.65169518e-316],
        [0.00000000e+000]],

       [[0.00000000e+000],
        [0.00000000e+000],
        [0.00000000e+000]],

       [[0.00000000e+000],
        [0.00000000e+000],
        [0.00000000e+000]]])

In [72]:
arr2

array([[[6.92263573e-310],
        [5.83562989e-316],
        [6.92250276e-310]],

       [[6.92250276e-310],
        [6.92250275e-310],
        [6.92250276e-310]],

       [[6.92250276e-310],
        [6.92250276e-310],
        [6.92250275e-310]]])

In [78]:
list1 = []
for i in range(100):
    list1.append(arr3)
list1 = np.asarray(list1)

In [79]:
list1.shape

(100, 3, 3, 2)

In [83]:
np.arange(4)

array([0, 1, 2, 3])

In [85]:
list2=[]

for i in range(100):
    list2.append(np.arange(4))

In [89]:
list2 = np.asarray(list2)

In [91]:
list2.shape

(100, 4)

## Display Data

In [5]:
def display_input_map(matrix):
    """
    This function is used to display maze with traversed path(highlighted)
    trajectory_path: path traversed by the agent so far
    matrix: maze of the problem
    """
    temp = matrix.copy()
    colormap = colors.ListedColormap(["green","black","white","blue"])
    plt.imshow(temp, cmap=colormap)
    plt.show()

In [6]:
def display_collected_data(X,y):
    for i in range(len(X)):
        display_input_map(np.asmatrix(X[i]))
        if y[i][0]==1:
            print("Up")
        elif y[i][1]==1:
            print("down")
        elif y[i][2]==1:
            print("left")
        elif y[i][3]==1:
            print("right")

In [347]:
sample_range = 100000
indices = list(np.random.choice(sample_range,10))
# print(indices)
sample_x = [X[index] for index in indices]
sample_y = [y[index] for index in indices]

In [348]:
# display_collected_data(sample_x,sample_y,4)

## Build Model

In [102]:
model = keras.models.Sequential([
    keras.Input(shape = (3,3,2)),
    keras.layers.Conv2D( filters = 64, kernel_size = (2,2), strides = (1,1),
                               padding = "valid", activation = 'relu' ),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(4, activation= None),
    keras.layers.Softmax()
])

In [103]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 2, 2, 64)          576       
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 100)               25700     
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 4)                 204       
                                                                 
 softmax_1 (Softmax)         (None, 4)                 0         
                                                                 
Total params: 31,530
Trainable params: 31,530
Non-trai

In [104]:
model.compile(loss = tf.keras.losses.binary_crossentropy, optimizer = tf.optimizers.Adam(),metrics=['accuracy'])

In [105]:
# [print(i.shape, i.dtype) for i in model.inputs]
# [print(o.shape, o.dtype) for o in model.outputs]
# [print(l.name, l.input_shape, l.dtype) for l in model.layers]

In [106]:
history = model.fit(list1, list2, epochs=10, batch_size=20,verbose=1, validation_split=0.2)

Epoch 1/10
4/4 [==============================] - 1s 44ms/step - loss: 1.9336 - accuracy: 0.5000 - val_loss: 1.9303 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 0s 6ms/step - loss: 1.9283 - accuracy: 1.0000 - val_loss: 1.9250 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 0s 7ms/step - loss: 1.9230 - accuracy: 1.0000 - val_loss: 1.9197 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 0s 7ms/step - loss: 1.9177 - accuracy: 1.0000 - val_loss: 1.9145 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 0s 6ms/step - loss: 1.9125 - accuracy: 1.0000 - val_loss: 1.9093 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 7ms/step - loss: 1.9073 - accuracy: 1.0000 - val_loss: 1.9041 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 9ms/step - loss: 1.9021 - accuracy: 1.0000 - val_loss: 1.8989 - val_accuracy: 1.0000
Epoch 8/10
4/4 [=================

In [357]:
model.save('./model_weights/model_1_v2.model')

INFO:tensorflow:Assets written to: ./model_weights/model_1_v2.model/assets


In [95]:
testing_model = model.evaluate(list1[:10],list2[:10],verbose=2)

1/1 - 0s - loss: 1.8837 - accuracy: 1.0000 - 92ms/epoch - 92ms/step


In [96]:
testing_acc = testing_model[1]*100
# Printing the accuracy
print('Test Accuracy - ', testing_acc,'%')

Test Accuracy -  100.0 %


In [97]:
testing_model

[1.8836597204208374, 1.0]

In [98]:
model.metrics_names

['loss', 'accuracy']

In [100]:
predictions = model.predict(list1[0:3])

In [361]:
X[0:1]

array([[[4, 3, 3, 3],
        [3, 3, 3, 3],
        [3, 3, 3, 3],
        [3, 3, 3, 3]]])

In [101]:
print(list2[:3],"\n",predictions)

[[0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]] 
 [[0.24004622 0.24008827 0.25988954 0.2599759 ]
 [0.24004622 0.24008827 0.25988954 0.2599759 ]
 [0.24004623 0.24008827 0.25988954 0.2599759 ]]


In [ ]:
tf.keras.losses.binary_crossentropy

In [ ]:
import matplotlib.pyplot as plt
plot_x = history.history['loss']
plot_y = list(np.arange(len(x)))

In [7]:
model_test = keras.models.load_model('model_weights/model_1_accuracy_100.model')

2021-12-13 00:43:09.080194: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-12-13 00:43:09.080238: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (coco): /proc/driver/nvidia/version does not exist
2021-12-13 00:43:09.080552: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
%run data_collection.ipynb

In [11]:
initialize(4,0.3)

In [12]:
final_grid = encode_grid(grid,[])
final_grid[0,0] = 4

In [13]:
np.asarray([final_grid.tolist()])

array([[[4, 3, 3, 3],
        [3, 3, 3, 2],
        [2, 3, 3, 3],
        [3, 3, 2, 3]]])

In [14]:
model_test.predict(np.asarray([final_grid.tolist()]))

array([[0.0000000e+00, 9.9634355e-01, 2.0873466e-12, 3.6564714e-03]],
      dtype=float32)